# Решение задачи:

Установим необходимый ряд библиотек:

In [ ]:
!pip install transformers ctransformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

Импортируем библиотеки:

In [ ]:
import os
from huggingface_hub import login, hf_hub_download
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer

Выполняем авторизацию в Hugging Face, указав заранее сгенерированный токен, полученный в настройках профиля:

In [ ]:
login(token="your_hf_t") #авторизация на Hugging Face

Выполняем скачивание квантованной модели:

In [ ]:
print("Скачивание модели Saiga (квантованная версия)...")
model_path = hf_hub_download(
    repo_id="IlyaGusev/saiga_mistral_7b_gguf", #указываем id репозитория
    filename="model-q8_0.gguf", #используем 8-битную квантованную версию
    local_dir="./models" #сохраняет локально в папку
)

Скачивание модели Saiga (квантованная версия)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-q8_0.gguf:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

Загрузим квантованную модель, указав дополнительные параметры:

In [ ]:
#загрузка модели
model = AutoModelForCausalLM.from_pretrained(
    model_path, #путь к локальному файлу модели
    model_type="mistral", #тип модели
    gpu_layers=50,  #количество слоев для загрузки на GPU
)

Выполним загрузку токенизатора. `use_fast=True` использует оптимизированную версию токенизатора на **Rust**:

In [ ]:
#загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(
    "IlyaGusev/saiga_mistral_7b",
    use_fast=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Укажем правила форматирования диалога, требуемые моделью **Saiga**:

In [ ]:
def format_chat_prompt(messages):
    prompt = ""
    for message in messages:
        prompt += f"<|im_start|>{message['role']}\n{message['content']}<|im_end|>\n"
    prompt += "<|im_start|>assistant\n"
    return prompt

Создадим список из 3 промптов для проверки работы модели. Также опишем процедуру генерации ответов и укажем необходимые параметры:

In [ ]:
print("\nГенерация ответов с помощью Saiga модели...\n")

prompts = [
    [
        {"role": "system", "content": "Ты - юморист, который является специалистом по анекдотам. Твои шутки должны быть позитивные и добрые, никого не должны оскорблять!"},
        {"role": "user", "content": "Напиши 1 короткую шутку про кота и собаку."}
    ],
    [
        {"role": "system", "content": "Ты - юморист, который является специалистом по анекдотам. Твои шутки должны быть позитивные и добрые, никого не должны оскорблять!"},
        {"role": "user", "content": "Напиши 1 короткую шутку про программиста и искуствнный интеллект."}
    ],
    [
        {"role": "system", "content": "Ты - юморист, который является специалистом по анекдотам. Твои шутки должны быть позитивные и добрые, никого не должны оскорблять!"},
        {"role": "user", "content": "Напиши 1 короткую шутку про язык программирования Python."}
    ]
]

for i, messages in enumerate(prompts, 1):
    prompt = format_chat_prompt(messages)

    response = model(
        prompt,
        max_new_tokens=200,
        temperature=0.2,
        top_p=0.95,
    )

    print(f"Промпт #{i}:")
    print(response.strip()) #убираем лишние пробелы
    print("-" * 50)


Генерация ответов с помощью Saiga модели...

Промпт #1:
Когда кот и собака решают, кто быстрее бегает, они обычно начинают с равных шансов. Но когда они приходят домой, то уже понимают, что кот всегда побеждает!
--------------------------------------------------
Промпт #2:
Программист: "Ай, я написал код, который может заменить любого человека!"
Искусственный Интеллект: "Ах да! Ты внезапно решил меня уволить?"
--------------------------------------------------
Промпт #3:
Вы знаете, что делает Python таким популярным среди программистов? Он всегда готов помочь!
--------------------------------------------------
